In [3]:
# future_predictions_comprehensive.py
# Comprehensive future prediction visualizations for "The Price of Progress"

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

def load_data():
    """Load and prepare the dataset"""
    try:
        df = pd.read_csv("../Final/final_clean_dataset.csv")
        print(f"✅ Data loaded: {len(df)} rows, {df['Country Name'].nunique()} countries")
        return df
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return None

def prepare_features(df):
    """Prepare features for predictive modeling"""
    df_clean = df.copy()
    
    # Create time-based features
    df_clean = df_clean.sort_values(['Country Name', 'Year'])
    
    # Lag features
    df_clean['Suicide_lag1'] = df_clean.groupby('Country Name')['Suicide_rate'].shift(1)
    df_clean['HDI_lag1'] = df_clean.groupby('Country Name')['HDI'].shift(1)
    df_clean['GDP_lag1'] = df_clean.groupby('Country Name')['GDP_per_capita'].shift(1)
    
    # Growth rates
    df_clean['HDI_growth'] = df_clean.groupby('Country Name')['HDI'].pct_change()
    df_clean['Suicide_growth'] = df_clean.groupby('Country Name')['Suicide_rate'].pct_change()
    df_clean['GDP_growth'] = df_clean.groupby('Country Name')['GDP_per_capita'].pct_change()
    
    # Polynomial features
    df_clean['HDI_sq'] = df_clean['HDI'] ** 2
    df_clean['GDP_sq'] = df_clean['GDP_per_capita'] ** 2
    df_clean['log_GDP'] = np.log(df_clean['GDP_per_capita'] + 1)
    
    # Encode categorical variables
    if 'continent' in df_clean.columns:
        le_continent = LabelEncoder()
        df_clean['continent_encoded'] = le_continent.fit_transform(df_clean['continent'].fillna('Unknown'))
    
    if 'income_group_auto' in df_clean.columns:
        le_income = LabelEncoder()
        df_clean['income_encoded'] = le_income.fit_transform(df_clean['income_group_auto'].fillna('Unknown'))
    
    # Select features for modeling
    feature_columns = [
        'HDI', 'HDI_sq', 'GDP_per_capita', 'log_GDP', 'GDP_sq',
        'Suicide_lag1', 'HDI_lag1', 'GDP_lag1',
        'HDI_growth', 'Suicide_growth', 'GDP_growth'
    ]
    
    # Add encoded features if available
    if 'continent_encoded' in df_clean.columns:
        feature_columns.append('continent_encoded')
    if 'income_encoded' in df_clean.columns:
        feature_columns.append('income_encoded')
    
    # Filter to available features
    available_features = [f for f in feature_columns if f in df_clean.columns]
    
    return df_clean, available_features

def train_ensemble_model(X, y):
    """Train ensemble of models for better predictions"""
    models = {
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42, max_depth=6),
        'Linear Regression': LinearRegression()
    }
    
    trained_models = {}
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        trained_models[name] = {
            'model': model,
            'mse': mean_squared_error(y_test, y_pred),
            'r2': r2_score(y_test, y_pred),
            'predictions': y_pred
        }
    
    # Select best model based on R² score
    best_model_name = max(trained_models.keys(), key=lambda x: trained_models[x]['r2'])
    print(f"🏆 Best model: {best_model_name} (R²: {trained_models[best_model_name]['r2']:.3f})")
    
    return trained_models[best_model_name]['model'], trained_models

def generate_scenarios():
    """Define different future scenarios"""
    scenarios = {
        'Pessimistic': {
            'hdi_growth': 0.001,  # Very slow HDI growth
            'gdp_growth': 0.005,  # Slow economic growth
            'stress_factor': 0.01, # High stress impact on mental health
            'color': 'red',
            'description': 'Stagnant development, high economic stress'
        },
        'Baseline': {
            'hdi_growth': 0.005,  # Moderate HDI growth
            'gdp_growth': 0.02,   # Moderate economic growth  
            'stress_factor': 0.005, # Moderate stress impact
            'color': 'blue',
            'description': 'Current trends continue'
        },
        'Optimistic': {
            'hdi_growth': 0.01,   # Rapid HDI growth
            'gdp_growth': 0.04,   # Strong economic growth
            'stress_factor': 0.002, # Low stress impact
            'color': 'green',
            'description': 'Accelerated development, mental health focus'
        }
    }
    return scenarios

def predict_future_trajectories(df, model, feature_columns, years=30):
    """Predict future trajectories for all countries under different scenarios"""
    scenarios = generate_scenarios()
    latest_year = df['Year'].max()
    future_years = list(range(latest_year + 1, latest_year + years + 1))
    
    all_predictions = []
    
    for scenario_name, scenario_params in scenarios.items():
        print(f"📈 Generating {scenario_name} scenario predictions...")
        
        for country in df['Country Name'].unique():
            country_data = df[df['Country Name'] == country].sort_values('Year')
            if len(country_data) < 2:
                continue
                
            latest = country_data.iloc[-1]
            
            # Initialize current values
            current_hdi = latest['HDI']
            current_suicide = latest['Suicide_rate']
            current_gdp = latest['GDP_per_capita']
            
            for year in future_years:
                # Project values based on scenario
                projected_hdi = min(1.0, current_hdi * (1 + scenario_params['hdi_growth']))
                projected_gdp = current_gdp * (1 + scenario_params['gdp_growth'])
                
                # Prepare features for prediction
                pred_features = {}
                for feature in feature_columns:
                    if feature == 'HDI':
                        pred_features[feature] = projected_hdi
                    elif feature == 'HDI_sq':
                        pred_features[feature] = projected_hdi ** 2
                    elif feature == 'GDP_per_capita':
                        pred_features[feature] = projected_gdp
                    elif feature == 'log_GDP':
                        pred_features[feature] = np.log(projected_gdp + 1)
                    elif feature == 'GDP_sq':
                        pred_features[feature] = projected_gdp ** 2
                    elif feature == 'Suicide_lag1':
                        pred_features[feature] = current_suicide
                    elif feature == 'HDI_lag1':
                        pred_features[feature] = current_hdi
                    elif feature == 'GDP_lag1':
                        pred_features[feature] = current_gdp
                    elif feature == 'HDI_growth':
                        pred_features[feature] = scenario_params['hdi_growth']
                    elif feature == 'GDP_growth':
                        pred_features[feature] = scenario_params['gdp_growth']
                    elif feature == 'Suicide_growth':
                        pred_features[feature] = 0  # Unknown for future
                    else:
                        # For encoded features, use latest value
                        pred_features[feature] = latest.get(feature, 0)
                
                # Create feature array
                feature_array = np.array([[pred_features.get(f, 0) for f in feature_columns]])
                
                # Get prediction
                try:
                    predicted_suicide = model.predict(feature_array)[0]
                    # Apply scenario-specific stress factor
                    predicted_suicide = max(0, predicted_suicide * (1 + scenario_params['stress_factor'] * (year - latest_year)))
                except:
                    predicted_suicide = current_suicide
                
                all_predictions.append({
                    'Year': year,
                    'Country Name': country,
                    'Continent': latest.get('continent', 'Unknown'),
                    'Income Group': latest.get('income_group_auto', 'Unknown'),
                    'Scenario': scenario_name,
                    'Predicted_Suicide_Rate': predicted_suicide,
                    'Projected_HDI': projected_hdi,
                    'Projected_GDP': projected_gdp,
                    'Current_HDI': latest['HDI'],
                    'Current_Suicide_Rate': latest['Suicide_rate']
                })
                
                # Update for next iteration
                current_hdi = projected_hdi
                current_gdp = projected_gdp
                current_suicide = predicted_suicide
    
    return pd.DataFrame(all_predictions)

# VISUALIZATION FUNCTIONS

def create_scenario_comparison_plot(predictions_df):
    """Create comparison of different scenarios"""
    # Aggregate by year and scenario
    scenario_trends = predictions_df.groupby(['Year', 'Scenario'])['Predicted_Suicide_Rate'].agg(['mean', 'std']).reset_index()
    
    fig = go.Figure()
    
    scenarios = predictions_df['Scenario'].unique()
    colors = {'Pessimistic': 'red', 'Baseline': 'blue', 'Optimistic': 'green'}
    
    # Define rgba colors manually to avoid conversion errors
    rgba_colors = {
        'Pessimistic': 'rgba(255, 0, 0, 0.2)',
        'Baseline': 'rgba(0, 0, 255, 0.2)',
        'Optimistic': 'rgba(0, 255, 0, 0.2)'
    }
    
    for scenario in scenarios:
        scenario_data = scenario_trends[scenario_trends['Scenario'] == scenario]
        
        fig.add_trace(go.Scatter(
            x=scenario_data['Year'],
            y=scenario_data['mean'],
            mode='lines',
            name=f'{scenario} Scenario',
            line=dict(color=colors.get(scenario, 'gray'), width=3),
            hovertemplate='<b>%{x}</b><br>Suicide Rate: %{y:.2f}<extra></extra>'
        ))
        
        # Add confidence interval
        fig.add_trace(go.Scatter(
            x=scenario_data['Year'],
            y=scenario_data['mean'] + scenario_data['std'],
            mode='lines',
            line=dict(width=0),
            showlegend=False,
            hoverinfo='skip'
        ))
        
        fig.add_trace(go.Scatter(
            x=scenario_data['Year'],
            y=scenario_data['mean'] - scenario_data['std'],
            mode='lines',
            line=dict(width=0),
            fill='tonexty',
            fillcolor=rgba_colors.get(scenario, 'rgba(128, 128, 128, 0.2)'),  # Fixed: use predefined rgba
            showlegend=False,
            hoverinfo='skip'
        ))
    
    # Add historical data for context
    historical_data = predictions_df.groupby('Year')['Current_Suicide_Rate'].mean().reset_index()
    latest_historical_year = historical_data['Year'].max()
    latest_historical_rate = historical_data[historical_data['Year'] == latest_historical_year]['Current_Suicide_Rate'].iloc[0]
    
    fig.add_trace(go.Scatter(
        x=[latest_historical_year, predictions_df['Year'].min()],
        y=[latest_historical_rate, latest_historical_rate],
        mode='lines',
        name='Historical Average',
        line=dict(color='black', width=2, dash='dash'),
        hovertemplate='Historical Average: %{y:.2f}<extra></extra>'
    ))
    
    fig.update_layout(
        title='Future Suicide Rate Projections: Scenario Comparison<br><sub>Global averages with confidence intervals</sub>',
        xaxis_title='Year',
        yaxis_title='Suicide Rate (per 100,000)',
        width=1200,
        height=600,
        hovermode='x unified'
    )
    
    return fig

def create_continental_predictions(predictions_df):
    """Create predictions grouped by continent"""
    continental_trends = predictions_df.groupby(['Year', 'Scenario', 'Continent'])['Predicted_Suicide_Rate'].mean().reset_index()
    
    fig = px.line(
        continental_trends,
        x='Year',
        y='Predicted_Suicide_Rate',
        color='Continent',
        line_dash='Scenario',
        title='Future Suicide Rate Projections by Continent and Scenario',
        labels={'Predicted_Suicide_Rate': 'Suicide Rate (per 100,000)'},
        width=1200,
        height=600
    )
    
    fig.update_layout(hovermode='x unified')
    return fig

def create_income_group_predictions(predictions_df):
    """Create predictions grouped by income level"""
    income_trends = predictions_df.groupby(['Year', 'Scenario', 'Income Group'])['Predicted_Suicide_Rate'].mean().reset_index()
    
    # Define order for income groups
    income_order = ['Low', 'Lower-Middle', 'Upper-Middle', 'High']
    income_trends['Income Group'] = pd.Categorical(income_trends['Income Group'], categories=income_order, ordered=True)
    income_trends = income_trends.sort_values('Income Group')
    
    fig = px.line(
        income_trends,
        x='Year',
        y='Predicted_Suicide_Rate',
        color='Income Group',
        line_dash='Scenario',
        title='Future Suicide Rate Projections by Income Group and Scenario',
        labels={'Predicted_Suicide_Rate': 'Suicide Rate (per 100,000)'},
        width=1200,
        height=600,
        color_discrete_sequence=px.colors.sequential.Viridis
    )
    
    fig.update_layout(hovermode='x unified')
    return fig

def create_country_spotlight(predictions_df, top_n=8):
    """Create spotlight on most concerning countries"""
    # Find countries with highest predicted increase in suicide rates
    latest_year = predictions_df['Year'].max()
    baseline_predictions = predictions_df[predictions_df['Scenario'] == 'Baseline']
    
    country_changes = []
    for country in baseline_predictions['Country Name'].unique():
        country_data = baseline_predictions[baseline_predictions['Country Name'] == country]
        if len(country_data) > 0:
            start_rate = country_data[country_data['Year'] == country_data['Year'].min()]['Predicted_Suicide_Rate'].iloc[0]
            end_rate = country_data[country_data['Year'] == latest_year]['Predicted_Suicide_Rate'].iloc[0]
            change = end_rate - start_rate
            country_changes.append({
                'Country': country,
                'Start_Rate': start_rate,
                'End_Rate': end_rate,
                'Change': change,
                'Percent_Change': (change / start_rate) * 100 if start_rate > 0 else 0,
                'Continent': country_data['Continent'].iloc[0]
            })
    
    concern_df = pd.DataFrame(country_changes).nlargest(top_n, 'Change')
    
    fig = px.bar(
        concern_df,
        x='Country',
        y='Change',
        color='Continent',
        title=f'Top {top_n} Countries with Highest Predicted Increase in Suicide Rates (Baseline Scenario)',
        labels={'Change': 'Predicted Increase in Suicide Rate'},
        hover_data=['Start_Rate', 'End_Rate', 'Percent_Change'],
        width=1200,
        height=600
    )
    
    fig.update_layout(xaxis_tickangle=-45)
    return fig, concern_df

def create_development_trajectory_plot(predictions_df):
    """Show how development level affects future predictions"""
    # Categorize countries by current HDI level
    predictions_df['Development_Level'] = pd.cut(
        predictions_df['Current_HDI'],
        bins=[0, 0.55, 0.70, 0.80, 1.0],
        labels=['Low', 'Medium', 'High', 'Very High']
    )
    
    dev_trends = predictions_df.groupby(['Year', 'Scenario', 'Development_Level'])['Predicted_Suicide_Rate'].mean().reset_index()
    
    fig = px.line(
        dev_trends,
        x='Year',
        y='Predicted_Suicide_Rate',
        color='Development_Level',
        line_dash='Scenario',
        facet_col='Scenario',
        title='Future Suicide Rate Projections by Current Development Level',
        labels={'Predicted_Suicide_Rate': 'Suicide Rate (per 100,000)'},
        width=1400,
        height=600
    )
    
    return fig

def create_prediction_heatmap(predictions_df):
    """Create heatmap of predictions by region and year"""
    # Aggregate by continent and year for baseline scenario
    heatmap_data = predictions_df[predictions_df['Scenario'] == 'Baseline'].pivot_table(
        values='Predicted_Suicide_Rate',
        index='Continent',
        columns='Year',
        aggfunc='mean'
    ).fillna(0)
    
    fig = px.imshow(
        heatmap_data,
        aspect='auto',
        title='Suicide Rate Predictions Heatmap: Baseline Scenario<br>by Continent and Year',
        color_continuous_scale='Reds',
        width=1200,
        height=600
    )
    
    fig.update_layout(
        xaxis_title='Year',
        yaxis_title='Continent'
    )
    
    return fig

def create_scenario_impact_map(predictions_df):
    """Create world map showing scenario impacts"""
    # Calculate difference between optimistic and pessimistic scenarios
    latest_year = predictions_df['Year'].max()
    
    scenario_comparison = []
    for country in predictions_df['Country Name'].unique():
        country_data = predictions_df[predictions_df['Country Name'] == country]
        pessimistic = country_data[(country_data['Scenario'] == 'Pessimistic') & (country_data['Year'] == latest_year)]
        optimistic = country_data[(country_data['Scenario'] == 'Optimistic') & (country_data['Year'] == latest_year)]
        
        if len(pessimistic) > 0 and len(optimistic) > 0:
            impact = pessimistic['Predicted_Suicide_Rate'].iloc[0] - optimistic['Predicted_Suicide_Rate'].iloc[0]
            scenario_comparison.append({
                'Country Name': country,
                'ISO3': country_data['ISO3'].iloc[0] if 'ISO3' in country_data.columns else '',
                'Impact': impact,
                'Continent': country_data['Continent'].iloc[0]
            })
    
    impact_df = pd.DataFrame(scenario_comparison)
    
    if 'ISO3' in impact_df.columns and impact_df['ISO3'].notna().any():
        fig = px.choropleth(
            impact_df,
            locations="ISO3",
            color="Impact",
            hover_name="Country Name",
            hover_data={"Impact": ":.2f", "Continent": True},
            color_continuous_scale="RdBu_r",
            title=f"Impact of Scenario Choice on Suicide Rates in {latest_year}<br>"
                  "(Pessimistic minus Optimistic Scenario)",
            projection="natural earth",
            width=1200,
            height=700
        )
        
        fig.update_layout(coloraxis_colorbar=dict(title="Rate Difference"))
    else:
        # Fallback to bar chart if no ISO3 codes
        fig = px.bar(
            impact_df.nlargest(20, 'Impact'),
            x='Country Name',
            y='Impact',
            color='Continent',
            title=f"Top 20 Countries with Largest Scenario Impact in {latest_year}",
            labels={'Impact': 'Pessimistic - Optimistic Difference'},
            width=1200,
            height=600
        )
        fig.update_layout(xaxis_tickangle=-45)
    
    return fig

def create_prediction_uncertainty_plot(predictions_df):
    """Show prediction uncertainty over time"""
    uncertainty_data = predictions_df.groupby(['Year', 'Scenario'])['Predicted_Suicide_Rate'].agg(['mean', 'std', 'count']).reset_index()
    
    fig = go.Figure()
    
    for scenario in predictions_df['Scenario'].unique():
        scenario_data = uncertainty_data[uncertainty_data['Scenario'] == scenario]
        
        fig.add_trace(go.Scatter(
            x=scenario_data['Year'],
            y=scenario_data['mean'],
            error_y=dict(type='data', array=scenario_data['std'], visible=True),
            mode='lines+markers',
            name=scenario,
            hovertemplate='<b>%{x}</b><br>Mean: %{y:.2f}<br>Std: %{error_y.array:.2f}<extra></extra>'
        ))
    
    fig.update_layout(
        title='Prediction Uncertainty Over Time<br><sub>Mean suicide rates with standard deviation error bars</sub>',
        xaxis_title='Year',
        yaxis_title='Suicide Rate (per 100,000)',
        width=1200,
        height=600,
        hovermode='x unified'
    )
    
    return fig

def main():
    """Main function to generate all prediction visualizations"""
    print("🚀 Starting comprehensive future prediction analysis...")
    
    # Load and prepare data
    df = load_data()
    if df is None:
        return
    
    df_clean, feature_columns = prepare_features(df)
    
    # Prepare training data
    training_data = df_clean.dropna(subset=feature_columns + ['Suicide_rate'])
    X = training_data[feature_columns]
    y = training_data['Suicide_rate']
    
    print(f"🤖 Training model with {len(X)} samples and {len(feature_columns)} features...")
    print(f"Features: {feature_columns}")
    
    # Train model
    model, all_models = train_ensemble_model(X, y)
    
    # Generate predictions
    print("🔮 Generating future predictions...")
    predictions_df = predict_future_trajectories(df_clean, model, feature_columns, years=30)
    
    print(f"✅ Generated {len(predictions_df)} future predictions")
    
    # Create all visualizations
    visualizations = {}
    
    print("\n📊 Creating visualizations...")
    
    # 1. Scenario Comparison
    print("1. Scenario comparison plot...")
    visualizations['scenario_comparison'] = create_scenario_comparison_plot(predictions_df)
    
    # 2. Continental Predictions
    print("2. Continental predictions...")
    visualizations['continental'] = create_continental_predictions(predictions_df)
    
    # 3. Income Group Predictions
    print("3. Income group predictions...")
    visualizations['income_groups'] = create_income_group_predictions(predictions_df)
    
    # 4. Country Spotlight
    print("4. Country spotlight...")
    visualizations['country_spotlight'], concern_df = create_country_spotlight(predictions_df)
    
    # 5. Development Trajectories
    print("5. Development trajectories...")
    visualizations['development_trajectories'] = create_development_trajectory_plot(predictions_df)
    
    # 6. Prediction Heatmap
    print("6. Prediction heatmap...")
    visualizations['heatmap'] = create_prediction_heatmap(predictions_df)
    
    # 7. Scenario Impact Map
    print("7. Scenario impact map...")
    visualizations['impact_map'] = create_scenario_impact_map(predictions_df)
    
    # 8. Uncertainty Plot
    print("8. Uncertainty visualization...")
    visualizations['uncertainty'] = create_prediction_uncertainty_plot(predictions_df)
    
    # Save all visualizations
    print("\n💾 Saving visualizations...")
    for name, fig in visualizations.items():
        fig.write_image(f"prediction_{name}.png", scale=2)
        fig.write_html(f"prediction_{name}.html")
        print(f"✅ Saved: prediction_{name}.png/.html")
    
    # Save prediction data
    predictions_df.to_csv("future_predictions_detailed.csv", index=False)
    concern_df.to_csv("high_risk_countries.csv", index=False)
    
    # Create model performance summary
    with open("prediction_analysis_summary.txt", "w") as f:
        f.write("FUTURE PREDICTION ANALYSIS SUMMARY\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Model Performance:\n")
        for model_name, metrics in all_models.items():
            f.write(f"- {model_name}: R² = {metrics['r2']:.3f}, MSE = {metrics['mse']:.3f}\n")
        f.write(f"\nBest Model: {max(all_models.keys(), key=lambda x: all_models[x]['r2'])}\n")
        f.write(f"\nPrediction Period: {predictions_df['Year'].min()} - {predictions_df['Year'].max()}\n")
        f.write(f"Countries Analyzed: {predictions_df['Country Name'].nunique()}\n")
        f.write(f"Total Predictions: {len(predictions_df)}\n\n")
        
        f.write("Key Findings:\n")
        baseline_end = predictions_df[(predictions_df['Scenario'] == 'Baseline') & (predictions_df['Year'] == predictions_df['Year'].max())]
        f.write(f"- Baseline 2050 Global Average: {baseline_end['Predicted_Suicide_Rate'].mean():.2f}\n")
        f.write(f"- Optimistic vs Pessimistic Difference: {baseline_end['Predicted_Suicide_Rate'].mean() - predictions_df[(predictions_df['Scenario'] == 'Optimistic') & (predictions_df['Year'] == predictions_df['Year'].max())]['Predicted_Suicide_Rate'].mean():.2f}\n\n")
        
        f.write("Highest Risk Countries (Baseline Scenario):\n")
        for _, row in concern_df.iterrows():
            f.write(f"- {row['Country']}: +{row['Change']:.2f} ({row['Percent_Change']:.1f}%)\n")
    
    print("\n📁 Files created:")
    print("   - prediction_*.png/.html (8 visualization files)")
    print("   - future_predictions_detailed.csv")
    print("   - high_risk_countries.csv") 
    print("   - prediction_analysis_summary.txt")
    
    print("\n🎉 Comprehensive future prediction analysis completed!")

if __name__ == "__main__":
    main()

🚀 Starting comprehensive future prediction analysis...
✅ Data loaded: 985 rows, 205 countries
🤖 Training model with 780 samples and 13 features...
Features: ['HDI', 'HDI_sq', 'GDP_per_capita', 'log_GDP', 'GDP_sq', 'Suicide_lag1', 'HDI_lag1', 'GDP_lag1', 'HDI_growth', 'Suicide_growth', 'GDP_growth', 'continent_encoded', 'income_encoded']
🏆 Best model: Linear Regression (R²: 0.157)
🔮 Generating future predictions...
📈 Generating Pessimistic scenario predictions...
📈 Generating Baseline scenario predictions...
📈 Generating Optimistic scenario predictions...
✅ Generated 18450 future predictions

📊 Creating visualizations...
1. Scenario comparison plot...
2. Continental predictions...
3. Income group predictions...
4. Country spotlight...
5. Development trajectories...
6. Prediction heatmap...
7. Scenario impact map...
8. Uncertainty visualization...

💾 Saving visualizations...
✅ Saved: prediction_scenario_comparison.png/.html
✅ Saved: prediction_continental.png/.html
✅ Saved: prediction_in